# Make Hint Feature Matrix

In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
from pandas import DataFrame
from collections import defaultdict
import scipy.stats as stats
import numpy as np

## Load Data

In [2]:
hw_df = pd.read_pickle('../data/pkl_data/hw_data.pkl')
cols = hw_df.columns
problem_list = list(set([(c[0],c[1]) for c in cols if len(c)==3]))
hw_df.head(1)

,"(2, 10, 1)","(2, 10, 2)","(2, 10, 3)","(2, 10, 4)","(2, 11, 1)","(2, 11, 2)","(2, 12, 1)","(2, 12, 2)","(2, 12, 3)","(2, 12, 4)",...,"(8, 3, 1)","(8, 3, 2)","(8, 3, 3)","(8, 3, 4)","(8, 4, 1)","(8, 4, 2)","(8, 4, 3)","(8, 5, 1)","(8, 5, 2)","(8, 6, 1)"
ID,,,,,,,,,,,,,,,,,,,,,
247,"[[2016-10-04 01:07:43, 1/12, 1, null]]","[[2016-10-04 01:07:43, 5/6, 1, null]]","[[2016-10-04 01:07:43, 1/12, 1, null]]","[[2016-10-04 01:07:43, 1, 1, null]]","[[2016-10-04 01:08:00, 4, 1, null]]","[[2016-10-04 01:08:00, 1/13, 1, null]]","[[2016-10-04 01:11:02, 1024, 1, null], [2016-1...","[[2016-10-04 01:11:02, 1/1024, 1, null], [2016...","[[2016-10-04 01:11:02, 10, 1, null], [2016-10-...","[[2016-10-04 01:11:02, 1/512, 0, null], [2016-...",...,"[[2016-11-17 01:10:21, 400, 0, 400/64], [2016-...","[[2016-11-17 01:19:27, (e^(-400/81))*((400/81)...","[[2016-11-17 01:19:27, (e^(-400/81))*((400/81)...","[[2016-11-17 00:22:58, 1-e^(-150), Please fin...","[[2016-11-17 02:01:33, 1/10, 1, 80.0*1/800.0],...","[[2016-11-17 02:01:33, (e^-(1/10))*((1/10)^2/(...","[[2016-11-17 02:01:33, 1-e^(-(1/10)*(1/10)), 0...","[[2016-11-17 02:22:33, (1/6.5)e^(-6.5*9), 0, e...","[[2016-11-17 03:20:07, e^(-(10/5.5))*(((10/5.5...","[[2016-11-17 02:33:13, e^(-2.1), 0, ((1/2) * e..."


In [3]:
assign_df = pd.read_pickle('../data/pkl_data/hint_assignment.pkl')
assign_df.head(3)

,ID,WEEK4,WEEK5,WEEK6,WEEK7,WEEK8,WEEK9,WEEK10
ID,,,,,,,,
75,75,1,0,1,0,1,0,0
233,233,0,1,1,0,1,1,0
134,134,0,1,1,0,1,1,0


In [4]:
features_df = pd.read_pickle('../data/pkl_data/features_df.pkl')
features_df.head(3)

,group,"(tries, 2, 10, 1)","(tries, 2, 10, 2)","(tries, 2, 10, 3)","(tries, 2, 10, 4)","(tries, 2, 11, 1)","(tries, 2, 11, 2)","(tries, 2, 12, 1)","(tries, 2, 12, 2)","(tries, 2, 12, 3)",...,"(correctness, 8, 6, 1)",ave_time_between_tries,att_problems,att_var,time_distri,num_correct_pros,ave_time_spent,sum_of_time_each_part,ave_exp_count,ID
ID,,,,,,,,,,,,,,,,,,,,,
247,Out(Strong),1,1,1,1,1,1,2,2,2,...,1,276.033654,208,9.189630,8.575699,0.944444,276.033654,276.033654,2.331839,247
336,Strong,0,0,0,0,0,0,0,0,0,...,1,275.284483,208,1.629456,9.855143,0.894495,275.284483,275.284483,1.954545,336
109,Out(Strong),1,1,1,1,1,1,1,1,1,...,1,481.220126,208,1.300198,5.796131,0.933884,481.220126,481.220126,2.378788,109


In [5]:
#new_fea_df = pd.read_pickle('pkl/features_df.pkl')
#eval_info = pd.read_pickle('eval_info.pkl')
#hint_log = pd.read_pickle('hint_log.pkl')

## Build Hint Dataframe

In [6]:
### Functions defined here
def get_hint_count(df, problem_list):
    """
    Get count of hint for four categories:
    1. all hints
    2. universal hints
    3. please write an expression hints
    4. conditional hints
    Return counts for each category.
    """
    hint_count_all = defaultdict(list)
    hint_count_uni = defaultdict(list)
    hint_count_exp = defaultdict(list)
    hint_count_con = defaultdict(list)
    for ind, row in df.iterrows():
        col_ind = 0
        row_fine = row.fillna(-111)
        single_count_all = defaultdict(int)
        single_count_uni = defaultdict(int)
        single_count_exp = defaultdict(int)
        single_count_con = defaultdict(int)
        col_list = row_fine.index
        for ro in row_fine:
            all_h = 0
            exp_h = 0
            uni_h = 0
            con_h = 0
            if col_ind >= len(row)-1:
                continue
            title = col_list[col_ind]
            if int(title[0]) < 4: #Skip assignment 1, 2, and 3.
                col_ind += 1
                continue
            pro_id = (title[0],title[1])
            if ro != -111:
                hint_c = [s[3]=='Hint' for s in ro]
                if sum(hint_c) > 0: # we have more than 1 hint
                    for r in ro:
                        if r[3]!='Hint' or r[1] == 'no attempt' or r[1] == '':
                            continue
                        if r[2][0] == " ": #conditional hints start with a space
                            con_h += 1
                            all_h += 1
                        elif r[2][:26] == "Please write an expression":
                            exp_h += 1
                            all_h += 1
                        elif r[2][:17] == "Can the answer be": #universal hints for negative number and fractional
                            uni_h += 1
                            all_h += 1
            single_count_all[pro_id] += all_h
            single_count_uni[pro_id] += uni_h
            single_count_exp[pro_id] += exp_h
            single_count_con[pro_id] += con_h
            col_ind += 1
        for p in problem_list:
            if int(p[0]) < 4:
                continue
            hint_count_all[p].append(single_count_all[p])
            hint_count_uni[p].append(single_count_uni[p])
            hint_count_exp[p].append(single_count_exp[p])
            hint_count_con[p].append(single_count_con[p])
    return hint_count_all, hint_count_uni, hint_count_exp, hint_count_con

def add_to_df(fea_name, fea_df, df):
    """Add a problem feature(feature with keys (week_id, problem_id, feature_name)) to a dataframe"""
    for k in fea_df.keys():
        df[(k[0], k[1], fea_name)] = fea_df[k]
    return df


def add_assign(assign, df):
    """Add assignment 1/0 to dataframe"""
    index_list = df.index.tolist()
    filtered_assign = assign.ix[index_list]
    df[('4','assign')] = filtered_assign['WEEK4']
    df[('5','assign')] = filtered_assign['WEEK5']
    df[('6','assign')] = filtered_assign['WEEK6']
    df[('7','assign')] = filtered_assign['WEEK7']
    df[('8','assign')] = filtered_assign['WEEK8']
    return df


def get_tries(df, problem_list):
    """Calculate number of tries for each problem"""
    tries_count = defaultdict(list)
    for ind, row in df.iterrows():
        col_ind = 0
        row_fine = row.fillna(-111)
        single_count = defaultdict(int)
        col_list = row_fine.index
        for ro in row_fine:
            if col_ind >= len(row)-1:
                continue
            title = col_list[col_ind]
            pro_id = (title[0],title[1])
            if ro == -111:
                single_count[pro_id] += 0
            elif len(ro) >= 0:
                tries_c = sum([r[3]!='Hint' for r in ro])
                single_count[pro_id] += tries_c
            col_ind += 1
        for p in problem_list:
            tries_count[p].append(single_count[p])
    return tries_count


def get_time_spent(HW_df, problem_list):
    """Calculate the time students spent on problems"""
    tim_diff = defaultdict(list)
    for ind, row in HW_df.iterrows():
        col_ind = 0
        row_fine = row.fillna(-111)
        single_count = defaultdict(int)
        col_list = row.index
        for ro in row_fine:
            if col_ind >= len(row)-1:
                continue
            title = col_list[col_ind]
            pro_id = (title[0],title[1])
            if ro == -111:
                single_count[pro_id] += 0
            elif len(ro) > 0:
                for r in range(len(ro)-1):
                    t1 = pd.to_datetime(ro[r][0])
                    t2 = pd.to_datetime(ro[r+1][0])
                    tmp = t2-t1
                    if tmp.seconds//60 < 10:
                        single_count[pro_id] += tmp.seconds
            col_ind += 1
        for p in problem_list:
            tim_diff[p].append(single_count[p])
    return tim_diff


def build_hint_df(HW_df, assign_df, features_df, hint_type, problem_list):
    """Build a new hint feature dataframe"""
    hints_df = DataFrame(HW_df.index)
    h_all, h_uni, h_exp, h_con = get_hint_count(HW_df, problem_list)
    if hint_type == "all":
        hint_count = h_all
    elif hint_type == "conditional":
        hint_count = h_con
    elif hint_type == "universal":
        hint_count = h_uni
    elif hint_type == "expressions":
        hint_count = h_exp
    hints_df = add_to_df('hint_count', hint_count, hints_df)
    hints_df = add_assign(assign_df, hints_df)
    hints_df = add_to_df('tries', get_tries(HW_df, problem_list), hints_df)
    #add_to_df('time_diff', get_time_diff(HW_df, problem_list), hints_df)
    add_to_df('time_spent', get_time_spent(HW_df, problem_list), hints_df)
    
    hints_df = hints_df.set_index('ID')
    return hints_df

In [7]:
hints_df_all = build_hint_df(hw_df, assign_df, features_df, "all", problem_list)
hints_df_all.head(3)

,"(7, 6, hint_count)","(4, 4, hint_count)","(8, 3, hint_count)","(5, 5, hint_count)","(7, 2, hint_count)","(4, 1, hint_count)","(8, 4, hint_count)","(7, 5, hint_count)","(4, 5, hint_count)","(5, 4, hint_count)",...,"(8, 2, time_spent)","(6, 6, time_spent)","(3, 1, time_spent)","(7, 3, time_spent)","(2, 12, time_spent)","(6, 5, time_spent)","(6, 4, time_spent)","(3, 5, time_spent)","(6, 3, time_spent)","(5, 1, time_spent)"
ID,,,,,,,,,,,,,,,,,,,,,
247,0,0,1,0,0,0,0,0,0,0,...,0,1160,0,3684,0,1044,0,0,6345,3444
336,0,0,0,3,0,0,0,0,0,0,...,0,960,268,0,0,0,177,0,8185,992
109,0,4,0,0,0,0,0,0,4,0,...,0,292,0,0,0,0,363,35,0,1867


In [8]:
pkl.dump(hints_df_all, open('../data/pkl_data/hint_feature_df.pkl', 'w'))